Example taken from Google ML Crash Course:
https://developers.google.com/machine-learning/crash-course/feature-crosses/encoding-nonlinearity

In [ ]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt 

import tensorflow as tf 

pd.options.display.max_rows = 10
pd.options.display.float_format = "{:.1f}".format

# pd.show_versions()

## Linear Problem

### Create the DataSet

50 data points per category

In [ ]:
N = 50

Create the random data set for sick trees

Scale and shift the (0, 1) range from np.random.random to (-5, 5) respectively (0, 5)

See also here: https://stackoverflow.com/questions/59389241/how-to-generate-a-random-float-in-range-1-1-using-numpy-random-rand

In [ ]:
max_x, min_x = 5, -5
range_size = (max_x - min_x)
sick_x = np.random.random([N]) * range_size + min_x

max_y, min_y = 0, -5
range_size = (max_y - min_y)
sick_y = np.random.random([N]) * range_size + min_y

Create the randomn data set for healthy trees

In [ ]:
max_x, min_x = 5, -5
range_size = (max_x - min_x)
healthy_x = np.random.random([N]) * range_size + min_x

max_y, min_y = 5, 0
range_size = (max_y - min_y)
healthy_y = np.random.random([N]) * range_size + min_y

Plot the data

In [ ]:
plt.figure(figsize=(10, 8))

plt.scatter(sick_x, sick_y)
plt.scatter(healthy_x, healthy_y)

plt.xlabel('x') 
plt.ylabel('y') 

plt.title("Training Data") 
plt.show() 

Create the Pandas DataFrame

In [ ]:
# combine the x and y arrays
sick_trees = np.vstack((sick_x, sick_y)).T
healthy_trees = np.vstack((healthy_x, healthy_x)).T

data_column_names = ['x', 'y']

df_sick = pd.DataFrame(data=sick_trees, columns=data_column_names)
df_sick['status'] = 1

df_healthy = pd.DataFrame(data=healthy_trees, columns=data_column_names)
df_healthy['status'] = 0

# append is deprecated since pandas 1.4.0
# train_df = df_sick.append(df_healthy, ignore_index=True)
train_df = pd.concat([df_sick, df_healthy], ignore_index=True)

# Shuffle the examples.
shuffled_train_df = train_df.reindex(np.random.permutation(train_df.index)) 
shuffled_train_df.head(100)

# Print the entire DataFrame
# print(df)

Build and Train Model Functions

In [ ]:
#@title Define the functions that build and train a model
def build_model(my_learning_rate):
  """Create and compile a simple linear regression model."""
  # Most simple tf.keras models are sequential.
  model = tf.keras.models.Sequential()

  # Add one linear layer to the model to yield a simple linear regressor.
  model.add(tf.keras.layers.Dense(units=1, input_shape=(2,)))

  # Compile the model topography into code that TensorFlow can efficiently
  # execute. Configure training to minimize the model's mean squared error. 
  model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=my_learning_rate),
                loss="mean_squared_error",
                metrics=[tf.keras.metrics.RootMeanSquaredError()])

  return model               


def train_model(model, x, y, my_epochs, 
                my_batch_size=None, my_validation_split=0.1):
  """Feed a dataset into the model in order to train it."""

  history = model.fit(x=x,
                      y=y,
                      batch_size=my_batch_size,
                      epochs=my_epochs,
                      validation_split=my_validation_split)

  # Gather the model's trained weight and bias.
  trained_weight = model.get_weights()[0]
  trained_bias = model.get_weights()[1]

  # The list of epochs is stored separately from the 
  # rest of history.
  epochs = history.epoch
  
  # Isolate the root mean squared error for each epoch.
  hist = pd.DataFrame(history.history)
  rmse = hist["root_mean_squared_error"]

  return epochs, rmse, history.history   

print("Defined the build_model and train_model functions.")

Define plotting functions

In [ ]:
#@title Define the plotting function

def plot_the_loss_curve(epochs, mae_training, mae_validation):
  """Plot a curve of loss vs. epoch."""

  plt.figure()
  plt.xlabel("Epoch")
  plt.ylabel("Root Mean Squared Error")

  plt.plot(epochs[1:], mae_training[1:], label="Training Loss")
  plt.plot(epochs[1:], mae_validation[1:], label="Validation Loss")
  plt.legend()
  
  # We're not going to plot the first epoch, since the loss on the first epoch
  # is often substantially greater than the loss for other epochs.
  merged_mae_lists = mae_training[1:] + mae_validation[1:]
  highest_loss = max(merged_mae_lists)
  lowest_loss = min(merged_mae_lists)
  delta = highest_loss - lowest_loss
  print(delta)

  top_of_y_axis = highest_loss + (delta * 0.05)
  bottom_of_y_axis = lowest_loss - (delta * 0.05)
   
  plt.ylim([bottom_of_y_axis, top_of_y_axis])
  plt.show()  

print("Defined the plot_the_loss_curve function.")

### Train

In [ ]:
# The following variables are the hyperparameters.
learning_rate = 0.01
epochs = 30
batch_size = 30

# Specify the feature and the label.
my_feature = "total_rooms"  # the total number of rooms on a specific city block.
my_label="median_house_value" # the median value of a house on a specific city block.
# That is, you're going to create a model that predicts house value based 
# solely on total_rooms.  

# Discard any pre-existing version of the model.
my_model = None

# Invoke the functions.
my_model = build_model(learning_rate)
epochs, rmse, history = train_model(my_model, 
                                        shuffled_train_df.drop('status', axis=1),
                                        shuffled_train_df[['status']],
                                        epochs, batch_size)

plot_the_loss_curve(epochs, history["root_mean_squared_error"], 
                    history["val_root_mean_squared_error"])

## Non-Linear Problem

Create the sample data set

Scale and shift the (0, 1) range from np.random.random to (-5, 5).

See also here: https://stackoverflow.com/questions/59389241/how-to-generate-a-random-float-in-range-1-1-using-numpy-random-rand

In [ ]:
N = 100
max_val, min_val = 5, -5
range_size = (max_val - min_val)  # 2
x = np.random.random([N]) * range_size + min_val
y = np.random.random([N]) * range_size + min_val

In [ ]:
plt.figure(figsize=(10, 8))

plt.scatter(x, y)

plt.xlabel('x') 
plt.ylabel('y') 

plt.title("Training Data") 
plt.show() 